In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv",
          "spark.sql.execution.arrow.enabled":"true"
         }
}

[I 2023-01-28 17:59:52,680.680 configure_magic] Magic cell payload received: {"conf": {"spark.pyspark.python": "python3", "spark.pyspark.virtualenv.enabled": "true", "spark.pyspark.virtualenv.type": "native", "spark.pyspark.virtualenv.bin.path": "/usr/bin/virtualenv", "spark.sql.execution.arrow.enabled": "true"}, "proxyUser": "assumed-role_fdp_blitvin-Isengard"}

[I 2023-01-28 17:59:52,680.680 configure_magic] Sending request to update kernel. Please wait while the kernel will be refreshed.


The kernel is successfully refreshed.

In [3]:
import pyEX as p
import requests
import json
import pandas as pd
import numpy as np
#import vectorbt as vbt
import matplotlib.pyplot as plt 
import itertools 
import datetime 
import awswrangler as wr
import vectorbt as vbt
import pyspark.sql.functions as py_f
import pyspark.sql.types as py_t
isSandbox = False
if isSandbox:
    token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
    iex_ver = 'sandbox'
else:
    #token ='pk_2e94555e43da4135a6032433c6b18fa5'
    token ='pk_79d147436c1349f3abbec37591323e52'
    iex_ver = 'stable'
pyEX_cl = p.Client(api_token=token)
timeframe='max'

In [4]:
if False:
    fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
    bitcoin_etf_symbols = ['BITO']
    commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
    fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
    equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
    'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                          'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                          'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
    sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
    bench_symbol=['SPY','QQQ','IVV']
    all_symbols =fx_etf_symbols+bitcoin_etf_symbols+commod_etf_symbols+fi_etf_symbols+equity_etf_symbols+bench_symbol+sectors
    #all_symbols
    columns_to_capture = ['close','open','symbol']
    data_folder = 's3://fsidatalake/eod_market_data/etf_trend_following/'
    parquet_file_out =f"{data_folder}/etf_market_data.parquet"

In [86]:
syms=pd.DataFrame([{'symbol':'SPY'}])
get_new_data_df(syms)


https://cloud.iexapis.com/stable/stock/SPY/chart/1mm?token=pk_79d147436c1349f3abbec37591323e52&chartIEXOnly=true


,date,minute,open,high,low,close,notional,numberOfTrades,symbol,marketOpen,marketHigh,marketLow,marketClose,marketNotional,marketNumberOfTrades
0,2022-12-29,09:30,379.670,381.770,379.120,381.755,32992658.0,518,SPY,379.6300,381.8099,376.8187,381.6000,2.994120e+09,62756
1,2022-12-29,10:00,381.720,382.885,381.395,382.230,36256662.5,571,SPY,381.6000,382.9200,376.8187,382.3000,3.355471e+09,58542
2,2022-12-29,10:30,382.630,383.220,382.420,383.200,29285614.5,513,SPY,382.3184,383.2600,382.3150,383.1101,2.021019e+09,45684
3,2022-12-29,11:00,383.100,383.200,382.275,382.840,22191317.0,352,SPY,383.1200,383.2501,376.8187,382.8600,1.469763e+09,31639
4,2022-12-29,11:30,382.860,384.230,382.860,383.925,18340663.5,328,SPY,382.8500,384.2400,382.8500,383.8900,1.398715e+09,34488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,2023-01-27,13:30,406.100,406.535,406.025,406.420,11172970.5,220,SPY,406.0400,406.5400,406.0100,406.4200,7.031249e+08,19035
254,2023-01-27,14:00,406.375,407.165,406.290,407.060,23026580.5,395,SPY,406.4150,407.2000,404.7662,407.0500,1.396809e+09,27657
255,2023-01-27,14:30,407.145,407.580,406.930,407.440,23336666.5,489,SPY,407.0450,408.0300,406.5000,407.4550,1.090953e+09,28500
256,2023-01-27,15:00,407.535,408.145,407.070,407.530,32651829.5,590,SPY,407.4600,408.1600,404.7662,407.6200,1.897571e+09,39453


In [87]:
#@py_f.pandas_udf("date timestamp, close double, open double, symbol string", py_f.PandasUDFType.GROUPED_MAP)
#@py_f.pandas_udf("symbol string", py_f.PandasUDFType.GROUPED_MAP)
def get_new_data_df(symbol):
    local_sym=str(symbol.values[0][0])
    try:
        hwm_l=highwatermark_pd_bc.value
        columns_to_capture=columns_to_capture_bc.value
        iex_base_url=iex_base_url_bc.value
        sym_pd = hwm_l.query(f"symbol=='{local_sym}'")
        if len(sym_pd)>0:
            max_date = sym_pd['max_date'].iloc[0]
        else:
            max_date = datetime.datetime.now()-datetime.timedelta(days=365*50)
        last_days=(datetime.datetime.now()-max_date).days

        pyEX_cl = p.Client(api_token=iex_token_bc.value)
        final_url = iex_base_url.format(local_sym,iex_token_bc.value,int(last_days))
        #'https://cloud.iexapis.com/stable/stock/{}/chart/max?token={}&chartIEXOnly=true&chartLast={}'
        print(final_url)
        res = requests.get(final_url)
        ret_pd = pd.DataFrame(json.loads(res.text))#[columns_to_capture]
        ret_pd=ret_pd.query(f"date>'{max_date.strftime('%Y-%m-%d')}'")
        ret_pd['date']=pd.to_datetime(ret_pd['date'])
        ret_pd['symbol']=local_sym
        #return(ret_pd[columns_to_capture].query(f"date>'{max_date}'"))
    except:
        ret_pd=pd.DataFrame([{'date': datetime.datetime.now()-datetime.timedelta(days=365*50),
                              'minute':None,
                              'open':None,
                              'high':None,
                              'low':None,
                              'close':None,
                              'notional':None,
                              'numberOfTrades':None,
                              'symbol': local_sym,
                              'marketOpen':None,
                              'marketHigh':None,
                              'marketLow':None,
                              'marketClose':None,
                              'marketNotional':None,
                              'marketNumberOfTrades':None
                             }])
    #return(ret_pd[columns_to_capture],max_date,hwm_l,local_sym,sym_pd)
    return(ret_pd[columns_to_capture])
    #return(pd.DataFrame([res.text],columns=['symbol']))
    
class MarketDataGateway():
    def __init__(self,isSandbox=False):
        self.fx_etf_symbols = ['FXA','FXY','FXE','FXB','UUP','FXC','FXF',]
        self.bitcoin_etf_symbols = ['BITO','BTF','XBTF','BITS','GBTC','BITW','BLOK','BLCN','LEGR','SPBC','BITQ','BKCH','DAPP','BTCFX']
        self.bitcoin_equity_proxy_symbols = ['COIN','MSTR']
        self.crypto_mining_symbols = ['RIOT','CAN','HUT','HIVE','MARA','BTCM','BTBT','BITF']
        self.commod_etf_symbols = ['USO','GLD','DBA','DBB','SLV',]
        self.fi_etf_symbols = ['IAGG','AGG','IHY','EMLC','HYG',]
        self.equity_etf_symbols = ['AMJ','AMLP','ARKF','ARKG','ARKK','ARKQ','ARKW','BOTZ','BUG','CGW','CIBR','COPX','CRBN','DRIV','EFV','EMLP','FBT','FDN','FIVG',
                'FIW','FTXG','GDX','GNR','GUNR','HACK','IAT','IBB','ICLN','IEO','IFRA','IGE','IGF','IGM','IGV','IHF','IHI','ITB','ITZ','IWD','IWN','IWS','IYT','JETS','KBWB',
                      'KOMP','KRE','KWEB','LCTU','LIT','MLPA',
                    'MLPX','MOO','NFRA','OIH','PABU','PAVE','PBW','PEJ','PHO','PICK','PPA','QCLN','REET','REMX','ROBO','SCHH','SIL','SKYY','SMH','SOXX','SRVR','TAN','URA','URNM',
                      'VUG','XAR','XBI','XHB','XME','XOP','XSD','QQQ','EWZ','AAXJ','VTI','EWJ','EWA','EWC','MCHI','EWU','EWQ','EWG','FM','EIS']
        self.sectors = ['XLC','XLY','XLP','XLE','XLF','XLV','XLI','XLB','XLRE','XLK','XLU']
        self.bench_symbol=['SPY','QQQ','IVV']
        self.all_symbols =self.fx_etf_symbols+\
                            self.bitcoin_etf_symbols+\
                            self.commod_etf_symbols+\
                            self.bitcoin_equity_proxy_symbols+\
                            self.crypto_mining_symbols+\
                            self.fi_etf_symbols+\
                            self.equity_etf_symbols+\
                            self.bench_symbol+\
                            self.sectors
        self.all_symbols_df = spark.createDataFrame(pd.DataFrame(self.all_symbols,columns=['symbol']))
        #all_symbols
        self.columns_to_capture = ['date','minute','open','high','low','close','notional','numberOfTrades','symbol'
                                   ,'marketOpen','marketHigh','marketLow','marketClose','marketNotional','marketNumberOfTrades']
        self.type_excl_field_map = {'date':'timestamp','minute':'string','symbol':'string'}
        self.pandas_udf_schema =[f"{i} double" for i in mdg.columns_to_capture if (i not in self.type_excl_field_map.keys()) ] +\
                                    [f"{i[0]} {i[1]}" for i in self.type_excl_field_map.items()]
        
        self.data_folder = 's3://fsidatalake/intraday_market_data/'
        self.parquet_file_out =f"{self.data_folder}/OHLC_30m.parquet"
        if isSandbox:
            self.iex_token ='Tpk_02dcd2036e7641b880dd4cbb01fa9c67'
            self.iex_ver = 'sandbox'
        else:
            #self.iex_token ='pk_2e94555e43da4135a6032433c6b18fa5' pk_79d147436c1349f3abbec37591323e52
            self.iex_token ='pk_79d147436c1349f3abbec37591323e52' 
            self.iex_ver = 'stable'
        self.iex_base_url = 'https://cloud.iexapis.com/'+self.iex_ver+'/stock/{}/chart/1mm?token={}&chartIEXOnly=true'
        
    def set_highwatermarks(self):
        try:
            self.old_df = spark.read.parquet(self.parquet_file_out)
        except:
            #self.old_df = spark.createDataFrame(pd.DataFrame([{'symbol':'----','date':datetime.datetime.now()-datetime.timedelta(days=365*50)}]))
            self.old_pd=self.all_symbols_df.toPandas()
            self.old_pd['date']=datetime.datetime.now()-datetime.timedelta(days=365*50)
            self.old_df= spark.createDataFrame(self.old_pd)
        self.highwatermark_pd = self.old_df.groupby('symbol').agg(py_f.max("date").alias("max_date"),
                             py_f.min("date").alias("min_date"),
                             (py_f.max("date")-py_f.min("date")).alias("date_length")
                            ).toPandas()
    def set_new_symbol_df(self):
        self.new_sym_df=self.all_symbols_df.groupby("symbol")\
                        .applyInPandas(get_new_data_df, schema= ','.join(self.pandas_udf_schema))\
                        .where('close is not null')
    def write_new_symbol_df(self):
        self.write_stats=self.new_sym_df.groupby('symbol').agg(py_f.count('symbol').alias('data_count'))
        self.new_sym_df.write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("append")\
            .parquet(self.parquet_file_out)
    def rewrite_if_dups(self):
        self.old_df_after_write = spark.read.parquet(self.parquet_file_out)
        self.dup_df = self.old_df_after_write.distinct().groupby('date','symbol').count()
        dup_count=self.dup_df.where('count>1').count()
        if dup_count>0:        
            print(f"found:{dup_count} dups; sample:{self.dup_df.show()}")
            self.old_df_after_write.groupby('date','symbol').agg(py_f.mean('close').alias('close'),py_f.mean('open').alias('open'))\
            .write\
            .option("header",True)\
            .partitionBy('symbol')\
            .mode("overwrite")\
            .parquet(self.parquet_file_out.replace('.parquet','_temp.parquet'))
        else:
            print(f"no dups")
        

mdg=MarketDataGateway()
mdg.set_highwatermarks()
# set bc vars
highwatermark_pd_bc =sc.broadcast(mdg.highwatermark_pd)
columns_to_capture_bc =sc.broadcast(mdg.columns_to_capture)
iex_token_bc =sc.broadcast(mdg.iex_token)
iex_base_url_bc =sc.broadcast(mdg.iex_base_url)
# set bc vars
mdg.set_new_symbol_df()
mdg.write_new_symbol_df()
print(mdg.write_stats.toPandas().sort_values(['symbol']))
#mdg.rewrite_if_dups()

    symbol  data_count
42    AAXJ         218
123    AGG         242
141    AMJ         104
66    AMLP         195
135   ARKF         203
..     ...         ...
110    XLV         247
77     XLY         243
27     XME         239
86     XOP         247
11     XSD          43

[142 rows x 2 columns]


In [ ]:
','.join(mdg.pandas_udf_schema)

In [98]:
sym='SPY'
sym_sample=spark.read.parquet(mdg.parquet_file_out).where(f"symbol=='{sym}'")

In [99]:
sym_sample = sym_sample.toPandas()

In [106]:
sym_sample['open_close']=((sym_sample['close']/sym_sample['open']-1)+(sym_sample['marketClose']/sym_sample['marketOpen']-1))/2
sym_sample_first_last=spy_sample_pd.query('minute=="09:30" or minute=="15:30"').dropna()
pd.pivot_table(sym_sample_first_last,values=['open_close','open','close','marketNumberOfTrades'],index=['date'],columns=['minute'])

close          marketNumberOfTrades               open           \
minute        09:30    15:30                09:30     15:30    09:30    15:30   
date                                                                            
2022-12-29  381.755  383.330              62756.0   66903.0  379.670  383.425   
2022-12-30  380.175  380.895              63498.0  127924.0  380.640  380.330   
2023-01-04  382.980  383.770              61337.0   77215.0  383.180  382.720   
2023-01-05  380.115  379.490              76033.0   74987.0  381.560  379.505   
2023-01-06  380.695  388.010              74447.0  106834.0  382.710  388.385   
2023-01-09  391.300  387.810              63824.0   79596.0  390.350  388.360   
2023-01-10  388.560  390.505              64248.0   66802.0  387.210  389.510   
2023-01-11  392.100  395.550              55727.0   77779.0  392.380  394.580   
2023-01-12  393.660  397.065              89551.0   76136.0  396.740  396.710   
2023-01-13  395.035  398.500              58959.0   73402.0  393.650  398.320   
2023-01-17  398.550  397.820              53145.0   64774.0  398.510  397.930   
2023-01-18  399.500  391.325              61074.0  100430.0  399.115  392.840   
2023-01-19  389.410  388.650              60825.0   87583.0  389.130  390.300   
2023-01-20  389.720  395.910              53481.0   85025.0  389.950  395.290   
2023-01-23  397.830  400.695              62189.0   77069.0  396.820  399.915   
2023-01-24  398.910  400.375              63718.0   56371.0  398.880  400.560   
2023-01-25  394.445  400.315              75526.0   70481.0  395.830  399.990   
2023-01-26  401.430  404.590              70077.0   72554.0  403.210  403.810   
2023-01-27  404.945  405.710              49726.0   80290.0  403.730  407.620   

           open_close            
minute          09:30     15:30  
date                             
2022-12-29   0.005340 -0.000183  
2022-12-30  -0.000887  0.003530  
2023-01-04  -0.000196  0.002652  
2023-01-05  -0.003989 -0.000323  
2023-01-06  -0.005080 -0.000927  
2023-01-09   0.002792 -0.001344  
2023-01-10   0.003293  0.002728  
2023-01-11  -0.000255  0.002393  
2023-01-12  -0.007613  0.000769  
2023-01-13   0.003169  0.000439  
2023-01-17   0.000138 -0.000364  
2023-01-18   0.001121 -0.003545  
2023-01-19   0.000270 -0.004138  
2023-01-20  -0.000785  0.001619  
2023-01-23   0.002703  0.001926  
2023-01-24  -0.000594 -0.000705  
2023-01-25  -0.003694  0.000881  
2023-01-26  -0.004224  0.002012  
2023-01-27   0.003165 -0.004735